# What Makes a Kaggler Valuable? 
Ever wondered what you should do to add some weight to your Data Science resume? Many of us already have a good notion of what is important to build a strong data science career. Of what is relevant to increase our compensation. But I personally, have never seen a systematic, data based, approach to this problem. That was the motivation of building a model to explain what makes a data scientist valuable to the market. Some results are pretty obvious, but many others might really help you boost your earnings.

* [See the article published on **Towards Data Science** that was written from this Kernel](https://towardsdatascience.com/what-makes-a-data-scientist-valuable-b723e6e814aa) 

* [We also deployed a model at AWS Lambda to predict the probability of earning more than U$100k per year. Check the code.](https://github.com/andresionek91/kaggle-top20-predictor)

## Learning how to increase your own compensation
We only could do this study because Kaggle has released the data from its [second annual Machine Learning and Data Science Survey](https://www.kaggle.com/kaggle/kaggle-survey-2018). The survey was live for one week in October 2018 and got a total of 23,859 responses. The results include raw numbers about who is working with data, what’s happening with machine learning in different industries, and the best ways for new data scientists to break into the field.

With access to that data, we wanted to understand what affects a Kaggler’s compensation (we are calling Kaggler anyone that answered the survey). Our idea was to give you precise insights of what is more valuable to the market, so you can stop spending time on things that won’t have a good ROI (return on investment) and speed up towards higher compensation. Following those insights, extracted from data, I hope one day you might find yourself laying down on a pile of money like Mr. Babineaux down here.

![](https://media.giphy.com/media/w1z2ilkWZagRG/giphy.gif)

## Considerations
1. We are assuming that respondents were honest and sincere in their answers.
2. This may not represent the whole universe of data professionals (it only has answers from Kaggle users), but it's a good proxy.

# Basic Cleaning
Survey answers are messy... Most survey softwares dont deliver the data on tidy format, and it is exactly the case of this survey. So we are going to have some really hard work to clean it. First lets just look at some personal data and clean it while we do our EDA.

In [1]:
import numpy as np 
import pandas as pd
from plotly.offline import init_notebook_mode

init_notebook_mode(connected=True)

try:
  import google.colab
  data_file_prefix = "https://raw.githubusercontent.com/slankas/VisualizationCode/master/kaggle/"
  import plotly.io as pio
  pio.renderers.default = 'colab'
except:
  data_file_prefix = ""


# Loading the multiple choices dataset, we will not look to the free form data on this study
mc = pd.read_csv(data_file_prefix + 'data/2018Survey/multipleChoiceResponses.csv', low_memory=False)

# Separating questions from answers
# This Series stores all questions
mcQ = mc.iloc[0,:]
# This DataFrame stores all answers
mcA = mc.iloc[1:,:]

In [2]:
# removing everyone that took less than 4 minutes or more than 600 minutes to answer the survey
less3 = mcA[round(mcA.iloc[:,0].astype(int) / 60) <= 4].index
mcA = mcA.drop(less3, axis=0)
more300 = mcA[round(mcA.iloc[:,0].astype(int) / 60) >= 600].index
mcA = mcA.drop(more300, axis=0)

# removing gender trolls, because we noticed from other kernels thata there are some ouliers here
gender_trolls = mcA[(mcA.Q1 == 'Prefer to self-describe') | (mcA.Q1 == 'Prefer not to say')].index
mcA = mcA.drop(list(gender_trolls), axis=0)

# removing student trolls, because a student won't make more than 250k a year.
student_trolls = mcA[((mcA.Q6 == 'Student') & (mcA.Q9 > '500,000+')) | \
                     ((mcA.Q6 == 'Student') & (mcA.Q9 > '400-500,000')) | \
                     ((mcA.Q6 == 'Student') & (mcA.Q9 > '300-400,000')) | \
                     ((mcA.Q6 == 'Student') & (mcA.Q9 > '250-300,000'))].index
mcA = mcA.drop(list(student_trolls), axis=0)

# dropping all NaN and I do not wish to disclose my approximate yearly compensation, because we are only interested in respondents that revealed their earnings
mcA = mcA[~mcA.Q9.isnull()].copy()
not_disclosed = mcA[mcA.Q9 == 'I do not wish to disclose my approximate yearly compensation'].index
mcA = mcA.drop(list(not_disclosed), axis=0)

We noticed that questions 1 through 9 are all about personal information of data scientists. So we are first focusing on them.

In [3]:
# Creating a table with personal data
personal_data = mcA.iloc[:,:13].copy()

# renaming columns
cols = ['survey_duration', 'gender', 'gender_text', 'age', 'country', 'education_level', 'undergrad_major', 'role', 'role_text',
        'employer_industry', 'employer_industry_text', 'years_experience', 'yearly_compensation']
personal_data.columns = cols

# Drop text and survey_duration columns 
personal_data.drop(['survey_duration', 'gender_text', 'role_text', 'employer_industry_text'], axis=1, inplace=True)

personal_data.head(3)

,gender,age,country,education_level,undergrad_major,role,employer_industry,years_experience,yearly_compensation
2,Male,30-34,Indonesia,Bachelor’s degree,Engineering (non-computer focused),Other,Manufacturing/Fabrication,5-10,"10-20,000"
3,Female,30-34,United States of America,Master’s degree,"Computer science (software engineering, etc.)",Data Scientist,I am a student,0-1,"0-10,000"
5,Male,22-24,India,Master’s degree,Mathematics or statistics,Data Analyst,I am a student,0-1,"0-10,000"


In [4]:
from pandas.api.types import CategoricalDtype

# transforming compensation into category type and ordening the values
categ = ['0-10,000', '10-20,000', '20-30,000', '30-40,000', '40-50,000',
         '50-60,000', '60-70,000', '70-80,000', '80-90,000', '90-100,000',
         '100-125,000', '125-150,000', '150-200,000', '200-250,000', '250-300,000',
         '300-400,000', '400-500,000', '500,000+']
cat_type = CategoricalDtype(categories=categ, ordered=True)
personal_data.yearly_compensation = personal_data.yearly_compensation.astype(cat_type)
# Doing this we are transforming the category "I do not wish to disclose my approximate yearly compensation" into NaN

# transforming age into category type and sorting the values
categ = ['18-21', '22-24', '25-29', '30-34', '35-39', '40-44', 
         '45-49', '50-54', '55-59', '60-69', '70-79', '80+']
cat_type = CategoricalDtype(categories=categ, ordered=True)
personal_data.age = personal_data.age.astype(cat_type)

# transforming years of experience into category type and sorting the values
categ = ['0-1', '1-2', '2-3', '3-4', '4-5', '5-10',
         '10-15', '15-20', '20-25', '25-30', '30+']
cat_type = CategoricalDtype(categories=categ, ordered=True)
personal_data.years_experience = personal_data.years_experience.astype(cat_type)

# transforming education level into category type and sorting the values
categ = ['No formal education past high school', 'Some college/university study without earning a bachelor’s degree',
         'Professional degree', 'Bachelor’s degree', 'Master’s degree', 'Doctoral degree', 'I prefer not to answer']
cat_type = CategoricalDtype(categories=categ, ordered=True)
personal_data.education_level = personal_data.education_level.astype(cat_type)

We have already dropped all participants that did not disclose their compensation. Let's see how many answers we have at each compensation, including NaNs (we expect to see none).

In [5]:
personal_data.yearly_compensation.value_counts(dropna=False, sort=False)

0-10,000       3652
10-20,000      1633
20-30,000      1177
30-40,000       886
40-50,000       796
50-60,000       767
60-70,000       631
70-80,000       587
80-90,000       429
90-100,000      486
100-125,000     728
125-150,000     473
150-200,000     404
200-250,000     152
250-300,000      64
300-400,000      39
400-500,000      16
500,000+         36
Name: yearly_compensation, dtype: int64

Now we want to create a numerical feature that describes compensation. I'm doing that by summing the lower and upper bound and then dividing by 2.  The highest range (500,000+) is summed with itself.

In [6]:
compensation = personal_data.yearly_compensation.str.replace(',', '').str.replace('500000\+', '500-500000').str.split('-')
personal_data['yearly_compensation_numerical'] = compensation.apply(lambda x: (int(x[0]) * 1000 + int(x[1]))/ 2) / 1000 # it is calculated in thousand dollars
print('Dataset Shape: ', personal_data.shape)
personal_data.head(3)

Dataset Shape:  (12956, 10)


,gender,age,country,education_level,undergrad_major,role,employer_industry,years_experience,yearly_compensation,yearly_compensation_numerical
2,Male,30-34,Indonesia,Bachelor’s degree,Engineering (non-computer focused),Other,Manufacturing/Fabrication,5-10,"10-20,000",15.0
3,Female,30-34,United States of America,Master’s degree,"Computer science (software engineering, etc.)",Data Scientist,I am a student,0-1,"0-10,000",5.0
5,Male,22-24,India,Master’s degree,Mathematics or statistics,Data Analyst,I am a student,0-1,"0-10,000",5.0


---
# EDA - Studying the problem: How personal data affects compensation?

Now that we have done a basic data cleaning, we are able to do some in depth EDA and ultimately build a model to predict the earnings of data scientists and find the most important features that affect compensation. Because we had 50 multiple choice questions, many of them with multiple answers, we will do an EDA only to analyse how personal data correlates to compensation. Other features will be used later on this study, on the modeling step.

**Note about splitting the data into train and test sets:** 
1. Ideally we would split the data into train and test sets before doing the Exploratory Data Analysis. This is good practice to avoid cognitive bias and overfitting the data. I'm not doing it here because I have already studied many of this dataset's kernels before start working on the data, so I'm probably already biased. 
2. We also have static data, I mean we are not getting any new answers from this survey. If we do the EDA and impute NaNs with the whole dataset, instead of just the training set, we shouldn't have any problem, because we are working with the full universe of data available. 

### Finding the Top 20% most well paid

In [7]:
# Finding the compensation that separates the Top 20% most welll paid from the Bottom 80%
top20flag = personal_data.yearly_compensation_numerical.quantile(0.8)
top20flag

85.0

In [8]:
# Creating a flag to identify who belongs to the Top 20%
personal_data['top20'] = personal_data.yearly_compensation_numerical > top20flag

# creating data for future mapping of values
top20 = personal_data.groupby('yearly_compensation', as_index=False)['top20'].min()

In [9]:
# Some helper functions to make our plots cleaner with Plotly
from plotly.offline import iplot
import plotly.graph_objs as go
from plotly import tools


def gen_xaxis(title):
    """
    Creates the X Axis layout and title
    """
    xaxis = dict(
            title=title,
            titlefont=dict(
                color='#AAAAAA'
            ),
            showgrid=False,
            color='#AAAAAA',
            )
    return xaxis


def gen_yaxis(title):
    """
    Creates the Y Axis layout and title
    """
    yaxis=dict(
            title=title,
            titlefont=dict(
                color='#AAAAAA'
            ),
            showgrid=False,
            color='#AAAAAA',
            )
    return yaxis


def gen_layout(charttitle, xtitle, ytitle, lmarg, h, annotations=None):  
    """
    Creates whole layout, with both axis, annotations, size and margin
    """
    return go.Layout(title=charttitle, 
                     height=h, 
                     width=800,
                     showlegend=False,
                     xaxis=gen_xaxis(xtitle), 
                     yaxis=gen_yaxis(ytitle),
                     annotations = annotations,
                     margin=dict(l=lmarg),
                    )


def gen_bars(data, color, orient):
    """
    Generates the bars for plotting, with their color and orient
    """
    bars = []
    for label, label_df in data.groupby(color):
        if orient == 'h':
            label_df = label_df.sort_values(by='x', ascending=True)
        if label == 'a':
            label = 'lightgray'
        bars.append(go.Bar(x=label_df.x,
                           y=label_df.y,
                           name=label,
                           marker={'color': label},
                           orientation = orient
                          )
                   )
    return bars


def gen_annotations(annot):
    """
    Generates annotations to insert in the chart
    """
    if annot is None:
        return []
    
    annotations = []
    # Adding labels
    for d in annot:
        annotations.append(dict(xref='paper', x=d['x'], y=d['y'],
                           xanchor='left', yanchor='bottom',
                           text= d['text'],
                           font=dict(size=13,
                           color=d['color']),
                           showarrow=False))
    return annotations


def generate_barplot(text, annot_dict, orient='v', lmarg=120, h=400):
    """
    Generate the barplot with all data, using previous helper functions
    """
    layout = gen_layout(text[0], text[1], text[2], lmarg, h, gen_annotations(annot_dict))
    fig = go.Figure(data=gen_bars(barplot, 'color', orient=orient), layout=layout)
    return iplot(fig)

In [10]:
# Counting the quantity of respondents per compensation
barplot = personal_data.yearly_compensation.value_counts(sort=False).to_frame().reset_index()
barplot.columns = ['yearly_compensation', 'qty']

# mapping back to get top 20% label
barplot = barplot.merge(top20, on='yearly_compensation')
barplot.columns = ['x', 'y', 'top20']

# apply color for top 20% and bottom 80%
barplot['color'] = barplot.top20.apply(lambda x: 'mediumaquamarine' if x else 'lightgray') 

# Create title and annotations
title_text = ['<b>How Much Does Kagglers Get Paid?</b>', 'Yearly Compensation (USD)', 'Quantity of Respondents']
annotations = [{'x': 0.06, 'y': 2200, 'text': '80% of respondents earn up to USD 90k','color': 'gray'},
              {'x': 0.51, 'y': 1100, 'text': '20% of respondents earn more than USD 90k','color': 'mediumaquamarine'}]

# call function for plotting
generate_barplot(title_text, annotations)

In [11]:
# creating masks to identify students and not students
is_student_mask = (personal_data['role'] == 'Student') | (personal_data['employer_industry'] == 'I am a student')
not_student_mask = (personal_data['role'] != 'Student') & (personal_data['employer_industry'] != 'I am a student')

# Counting the quantity of respondents per compensation (where is student)
barplot = personal_data[is_student_mask].yearly_compensation.value_counts(sort=False).to_frame().reset_index()
barplot.columns = ['yearly_compensation', 'qty']

# mapping back to get top 20%
barplot.columns = ['x', 'y',]
barplot['highlight'] = barplot.x != '0-10,000'

# applying color
barplot['color'] = barplot.highlight.apply(lambda x: 'lightgray' if x else 'crimson')

# title and annotations
title_text = ['<b>Do Students Get Paid at All?</b><br><i>only students</i>', 'Yearly Compensation (USD)', 'Quantity of Respondents']
annotations = [{'x': 0.06, 'y': 1650, 'text': '75% of students earn up to USD 10k','color': 'crimson'}]

# ploting
generate_barplot(title_text, annotations)

--- 
# What If We Remove Students From Our Data?
We have seen that students aren't usually remunerated: 76% of them earn up to USD 10k. Because they have very low compensation (and aren't actually working), they are probably biasing the data towards lower compensation. That is why we are removing them from the rest of EDA.

In [12]:
# Finding the compensation that separates the Top 20% most welll paid from the Bottom 80% (without students)
top20flag_no_students = personal_data[not_student_mask].yearly_compensation_numerical.quantile(0.8)
top20flag_no_students

95.0

In [13]:
# Creating a flag for Top 20% when there are no students in the dataset
personal_data['top20_no_students'] = personal_data.yearly_compensation_numerical > top20flag_no_students

# creating data for future mapping of values
top20 = personal_data[not_student_mask].groupby('yearly_compensation', as_index=False)['top20_no_students'].min()

# Counting the quantity of respondents per compensation (where is not student)
barplot = personal_data[not_student_mask].yearly_compensation.value_counts(sort=False).to_frame().reset_index()
barplot.columns = ['yearly_compensation', 'qty']

# mapping back to get top 20%
barplot = barplot.merge(top20, on='yearly_compensation')
barplot.columns = ['x', 'y', 'top20']
barplot['color'] = barplot.top20.apply(lambda x: 'mediumaquamarine' if x else 'lightgray')

title_text = ['<b>How Much Does Kagglers Get Paid?</b><br><i>without students</i>', 'Yearly Compensation (USD)', 'Quantity of Respondents']
annotations = [{'x': 0.06, 'y': 1600, 'text': '80% of earn up to USD 100k','color': 'gray'},
              {'x': 0.56, 'y': 800, 'text': '20% of earn more than USD 100k','color': 'mediumaquamarine'}]

generate_barplot(title_text, annotations)

### Are there any gender difference between the top 20% most well paid?
Unfortunatelly we still have differences in payment due to gender. This gets very noticeable when we compare the top 20% most well paid men and women.

In [14]:
# Creating a helper function to generate lineplot
def gen_lines(data, colorby):
    """
    Generate the lineplot with data
    """
    if colorby == 'top20': 
        colors = {False: 'lightgray',
                  True: 'mediumaquamarine'}
    else:
        colors = {False: 'lightgray',
                  True: 'deepskyblue'}

    traces = []
    for label, label_df in data.groupby(colorby):
        traces.append(go.Scatter(
                    x=label_df.x,
                    y=label_df.y,
                    mode='lines+markers+text',
                    line={'color': colors[label], 'width':2},
                    connectgaps=True,
                    text=label_df.y.round(),
                    hoverinfo='none',
                    textposition='top center',
                    textfont=dict(size=12, color=colors[label]),
                    marker={'color': colors[label], 'size':8},
                   )
                   )
    return traces

In [15]:
# Grouping data to get compensation per gender of Top20% and Bottom 80%
barplot = personal_data[not_student_mask].groupby(['gender', 'top20_no_students'], as_index=False)['yearly_compensation_numerical'].mean()
barplot = barplot[(barplot['gender'] == 'Female') | (barplot['gender'] == 'Male')]
barplot.columns = ['x', 'gender', 'y']

# Creates annotations
annot_dict = [{'x': 0.05, 'y': 180, 'text': 'The top 20% men are almost 12% better paid than the top 20% woman','color': 'deepskyblue'},
              {'x': 0.05, 'y': 60, 'text': 'At the bottom 20% there is almost no difference in payment','color': 'gray'}]

# Creates layout
layout = gen_layout('<b>What is the gender difference in compensation at the top 20%?</b><br><i>without students</i>', 
                    'Gender', 
                    'Average Yearly Compensation (USD)',
                    120, 
                    400,
                    gen_annotations(annot_dict)
                    )
# Make plot
fig = go.Figure(data=gen_lines(barplot, 'gender'), 
                layout=layout)
iplot(fig, filename='color-bar')

### Should you get formal education?
To earn more in this field you have either to go all the way up the formal education and get a Doctoral Degree, or you just don't get any formal education at all. It obviously doesn't mean that you should quit college, but that you are problable better off studying by yourself than attending a post-gratuation program.

In [16]:
# Calculates compensation per education level
barplot = personal_data[not_student_mask].groupby(['education_level'], as_index=False)['yearly_compensation_numerical'].mean()
barplot['no_college'] = (barplot.education_level == 'No formal education past high school') | \
                        (barplot.education_level == 'Doctoral degree')

# creates a line break for better visualisation
barplot.education_level = barplot.education_level.str.replace('study without', 'study <br> without')

barplot.columns = ['y', 'x', 'no_college']
barplot = barplot.sort_values(by='x', ascending=True)
barplot['color'] = barplot.no_college.apply(lambda x: 'coral' if x else 'a')

# Add title and annotations
title_text = ['<b>Impact of Formal Education on Compenstaion</b><br><i>without students</i>', 'Average Yearly Compensation (USD)', 'Level of Education']
annotations = []

generate_barplot(title_text, annotations, orient='h', lmarg=300)

### Which industry should you target?
If you concentrate your efforts on some industry specific problems you'll eventually get hired by them. Bellow we show the top 5 industries, and their average yearly compensation, compared to all others sectors. Choose wisely!

In [17]:
# Calculates compensation per industry
barplot = personal_data[not_student_mask].groupby(['employer_industry'], as_index=False)['yearly_compensation_numerical'].mean()

# Flags the top 5 industries to add color
barplot['best_industries'] = (barplot.employer_industry == 'Medical/Pharmaceutical') | \
                             (barplot.employer_industry == 'Insurance/Risk Assessment') | \
                             (barplot.employer_industry == 'Military/Security/Defense') | \
                             (barplot.employer_industry == 'Hospitality/Entertainment/Sports') | \
                             (barplot.employer_industry == 'Accounting/Finance')

barplot.columns = ['y', 'x', 'best_industries']
barplot = barplot.sort_values(by='x', ascending=True)
barplot['color'] = barplot.best_industries.apply(lambda x: 'darkgoldenrod' if x else 'a')

title_text = ['<b>Average Compensation per Industry | Top 5 in Color</b><br><i>without students</i>', 'Average Yearly Compensation (USD)', 'Industry']
annotations = []

generate_barplot(title_text, annotations, orient='h', lmarg=300, h=600)

### Should You Aim at the C-level?
It's obvious that a C-level compensation is much higher than an analyst's. But how much? Almost 3x. Also, managerial levels have better compensation. Makes sense, no?

In [18]:
# Calculates compensation per role
barplot = personal_data[not_student_mask].groupby(['role'], as_index=False)['yearly_compensation_numerical'].mean()

# Flags the top 5 roles to add color
barplot['role_highlight'] = (barplot.role == 'Data Scientist') | \
                        (barplot.role == 'Product/Project Manager') | \
                        (barplot.role == 'Consultant') | \
                        (barplot.role == 'Data Journalist') | \
                        (barplot.role == 'Manager') | \
                        (barplot.role == 'Principal Investigator') | \
                        (barplot.role == 'Chief Officer')

barplot.columns = ['y', 'x', 'role_highlight']
barplot = barplot.sort_values(by='x', ascending=True)
barplot['color'] = barplot.role_highlight.apply(lambda x: 'mediumvioletred' if x else 'lightgray')

title_text = ['<b>Average Compensation per Role | Top 7 in Color</b><br><i>without students</i>', 'Average Yearly Compensation (USD)', 'Job Title']
annotations = [{'x': 0.6, 'y': 11.5, 'text': 'The first step into the ladder<br>of better compensation is<br>becoming a Data Scientist','color': 'mediumvioletred'}]

generate_barplot(title_text, annotations, orient='h', lmarg=300, h=600)

### Which countries pay more?
Does living on certain contries impact the average compensation you get? Below we show how much, on average, you should expect to earn in each country.

In [19]:
# Replacing long country names
personal_data.country = personal_data.country.str.replace('United Kingdom of Great Britain and Northern Ireland', 'United Kingdom')
personal_data.country = personal_data.country.str.replace('United States of America', 'United States')
personal_data.country = personal_data.country.str.replace('I do not wish to disclose my location', 'Not Disclosed')
personal_data.country = personal_data.country.str.replace('Iran, Islamic Republic of...', 'Iran')
personal_data.country = personal_data.country.str.replace('Hong Kong \(S.A.R.\)', 'Hong Kong')
personal_data.country = personal_data.country.str.replace('Viet Nam', 'Vietnam')
personal_data.country = personal_data.country.str.replace('Republic of Korea', 'South Korea')

# Calculates compensation per country
barplot = personal_data[not_student_mask].groupby(['country'], as_index=False)['yearly_compensation_numerical'].mean()

# Flags the top 10 countries to add color
barplot['country_highlight'] = (barplot.country == 'United States') | \
                               (barplot.country == 'Switzerland') | \
                               (barplot.country == 'Australia') | \
                               (barplot.country == 'Israel') | \
                               (barplot.country == 'Denmark') | \
                               (barplot.country == 'Canada') | \
                               (barplot.country == 'Hong Kong') | \
                               (barplot.country == 'Norway') | \
                               (barplot.country == 'Ireland') | \
                               (barplot.country == 'United Kingdom')

barplot.columns = ['y', 'x', 'country_highlight']
barplot = barplot.sort_values(by='x', ascending=True)
barplot['color'] = barplot.country_highlight.apply(lambda x: 'mediumseagreen' if x else 'lightgray')

title_text = ['<b>Average Compensation per Country - Top 10 in Color</b><br><i>without students</i>', 'Average Yearly Compensation (USD)', 'Country']
annotations = []

generate_barplot(title_text, annotations, orient='h', lmarg=300, h=1200)

We see that countries that have a higher cost of living are showing up at the top, paying more. Let's try to divide the average compensation by the cost of living to normalize this feature? We found a ranking of cost of living per country at [Expatistan.com](https://www.expatistan.com/cost-of-living/country/ranking) on Nov. 15th 2018. This source also provide a price index for each country, that is calculated as described below: 

> To calculate each country's Price Index value, we start by assigning a value of 100 to a central reference country (that happens to be the Czech Republic). Once the reference point has been established, the Price Index value of every other country in the database is calculated by comparing their cost of living to the cost of living in the Czech Republic.
Therefore, if a country has a Price Index of 134, that means that living there is 34% more expensive than living in the Czech Republic. Source: [Expatistan.com](https://www.expatistan.com/cost-of-living/country/ranking)

In [20]:
# Loading the cost of living
cost_living = pd.read_csv(data_file_prefix + 'data/cost-of-living.csv')
cost_living.columns = ['ranking', 'country', 'price_index']
cost_living.head()

,ranking,country,price_index
0,1,Bermuda,317
1,2,Cayman Islands,310
2,3,Switzerland,260
3,4,Hong Kong,250
4,5,Bahamas,234


In [21]:
# joining both tables
personal_data = personal_data.merge(cost_living, on='country') # doing an inner join to avoid nans on normalized compensation

# calculating the normalized compensation
personal_data['normalized_compensation'] = personal_data.yearly_compensation_numerical / personal_data.price_index * 10
personal_data['normalized_compensation'] = personal_data['normalized_compensation'].round() * 10

In [22]:
# recreating masks
is_student_mask = (personal_data['role'] == 'Student') | (personal_data['employer_industry'] == 'I am a student')
not_student_mask = (personal_data['role'] != 'Student') & (personal_data['employer_industry'] != 'I am a student')

In [23]:
# Calculates compensation per country
barplot = personal_data[not_student_mask].groupby(['country'], as_index=False)['normalized_compensation'].mean()

# Flags the top 10 countries to add color
barplot['country_highlight'] = (barplot.country == 'United States') | \
                               (barplot.country == 'Australia') | \
                               (barplot.country == 'Israel') | \
                               (barplot.country == 'Switzerland') | \
                               (barplot.country == 'Canada') | \
                               (barplot.country == 'Tunisia') | \
                               (barplot.country == 'Germany') | \
                               (barplot.country == 'Denmark') | \
                               (barplot.country == 'Colombia') | \
                               (barplot.country == 'South Korea')

barplot.columns = ['y', 'x', 'country_highlight']
barplot = barplot.sort_values(by='x', ascending=True)
barplot['color'] = barplot.country_highlight.apply(lambda x: 'mediumseagreen' if x else 'lightgray')

title_text = ['<b>Normalized Average Compensation per Country - Top 10 in Color</b><br><i>without students</i>', 
              'Normalized Average Yearly Compensation (USD)', 'Country']
annotations = []

generate_barplot(title_text, annotations, orient='h', lmarg=300, h=1200)

We see that compensation is much smoother when we divide it by the cost of living. By livinng in most countries around the world, you should get almost the same compensation on average (between USD 30k and 40k per year). A few countries pay above the average (United States pays better than any other country), and other few countries pay below the average. 

## Top 20% by considering each country cost of living
Let's define the top 20% again, now based on the normalized compensation to see it in a chart. 

In [24]:
# Defining the threshold for top 20% most paid
top20_tresh = personal_data.normalized_compensation.quantile(0.8)
personal_data['top20'] = personal_data.normalized_compensation > top20_tresh

# creating data for future mapping of values
top20 = personal_data.groupby('normalized_compensation', as_index=False)['top20'].min()

# Calculates respondents per compensation
barplot = personal_data.normalized_compensation.value_counts(sort=False).to_frame().reset_index()
barplot.columns = ['normalized_compensation', 'qty']

# mapping back to get top 20% and 50%
barplot = barplot.merge(top20, on='normalized_compensation')
barplot.columns = ['x', 'y', 'top20']
barplot['color'] = barplot.top20.apply(lambda x: 'mediumaquamarine' if x else 'lightgray')

title_text = ['<b>How Much Does Kagglers Get Paid?<br></b><i>normalized by cost of living</i>', 'Normalized Yearly Compensation', 'Quantity of Respondents']
annotations = [{'x': 0.1, 'y': 1000, 'text': '20% Most well paid','color': 'mediumaquamarine'}]

generate_barplot(title_text, annotations)